In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
import time
import pyperclip

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####  요약
- content/tag 특수문자 제거
-like 전처리(이상값 대체 / int 형식으로 변환 / 결측치 대체)

## 1. 데이터 정보 확인

In [3]:
pasta=pd.read_csv("./data/pasta_result1518.csv", index_col = 0)
# pasta.columns = ["UserId", "Contents", "Tags", "Like"]
pasta.head()

,Contents,Like,ID,Tags
0,새우장 키타라 파스타 집에서 냉동새우로 새우장을 담가봤어요!제가 만들었지만 너무 맛...,204,jooboossong,"['#우리의식탁', '#라이프스타일러', '#파스타', '#파스타레시피', '#파스..."
1,@chef_seokdeok안녕하세요. 이태리에서 ??요리학교를 나와 지금까지 이태리...,lerice_cake님 여러 명이 좋아합니,jyourpso1028,"['#파스타', '#생면파스타', '#연남맛집', '#연남동맛집', '#홍대맛집',..."
2,@chef_seokdeok안녕하세요. 이태리에서 ??요리학교를 나와 지금까지 이태리...,seunp0203님 여러 명이 좋아합니,uoorspn,"['#파스타', '#생면파스타', '#연남맛집', '#연남동맛집', '#홍대맛집',..."
3,@chef_seokdeok안녕하세요. 이태리에서 ??요리학교를 나와 지금까지 이태리...,0721.pdkd님 여러 명이 좋아합니,jsytxmap,"['#파스타', '#생면파스타', '#연남맛집', '#연남동맛집', '#홍대맛집',..."
4,@chef_seokdeok안녕하세요. 이태리에서 ??요리학교를 나와 지금까지 이태리...,omsey1102님 여러 명이 좋아합니,jpskanwa0118,"['#파스타', '#생면파스타', '#연남맛집', '#연남동맛집', '#홍대맛집',..."


In [4]:
pasta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1518 entries, 0 to 1517
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Contents  1518 non-null   object
 1   Like      1517 non-null   object
 2   ID        1518 non-null   object
 3   Tags      1518 non-null   object
dtypes: object(4)
memory usage: 59.3+ KB


In [23]:
pasta[pasta["Like"].isna()==True]

,Contents,Like,ID,Tags
798,수비드 부채살과콰몰리또띠아 아주 맛있었던 조합#수비드 #과콰몰리#집밥 #셰프 #쉐프...,NaN,pesto_chef,"['#수비드', '#과콰몰리', '#집밥', '#셰프', '#쉐프', '#주부', ..."


## 2. 전처리

In [14]:
pasta1 = pasta.copy()

#### 기본 전처리
- content/tag 특수문자 제거

In [15]:
# contents 특수기호 및 문자 제거
pasta1["Contents"] = pasta1["Contents"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

# # tags # 제외한 특수 기호 제거
pasta1["Tags"] = pasta1["Tags"].str.strip("]['")
pasta1["Tags"] = pasta1["Tags"].str.replace("'", repl=r"", regex=True)

In [16]:
pasta1.head()

,Contents,Like,ID,Tags
0,새우장 키타라 파스타 집에서 냉동새우로 새우장을 담가봤어요 제가 만들었지만 너무 맛...,204,jooboossong,"#우리의식탁, #라이프스타일러, #파스타, #파스타레시피, #파스타만들기, #새우장..."
1,chef_seokdeok안녕하세요 이태리에서 요리학교를 나와 지금까지 이태리...,lerice_cake님 여러 명이 좋아합니,jyourpso1028,"#파스타, #생면파스타, #연남맛집, #연남동맛집, #홍대맛집, #연트럴파크맛집, ..."
2,chef_seokdeok안녕하세요 이태리에서 요리학교를 나와 지금까지 이태리...,seunp0203님 여러 명이 좋아합니,uoorspn,"#파스타, #생면파스타, #연남맛집, #연남동맛집, #홍대맛집, #연트럴파크맛집, ..."
3,chef_seokdeok안녕하세요 이태리에서 요리학교를 나와 지금까지 이태리...,0721.pdkd님 여러 명이 좋아합니,jsytxmap,"#파스타, #생면파스타, #연남맛집, #연남동맛집, #홍대맛집, #연트럴파크맛집, ..."
4,chef_seokdeok안녕하세요 이태리에서 요리학교를 나와 지금까지 이태리...,omsey1102님 여러 명이 좋아합니,jpskanwa0118,"#파스타, #생면파스타, #연남맛집, #연남동맛집, #홍대맛집, #연트럴파크맛집, ..."


In [ ]:
# pasta1.to_csv("./data/pasta_preprocessing.csv")

- like 전처리(이상값 대체 / int 형식으로 변환 / 결측치 0)

In [20]:
pasta1["Like"] = pasta1["Like"].fillna(0)

In [24]:
pasta1["Like"].isna().value_counts()   # 결측치 0으로 대체 완료

False    1518
Name: Like, dtype: int64

In [27]:
like_num = pasta1[~(pasta1["Like"].str.contains("좋아합니")|pasta1["Like"].str.contains("눌러보세"))]["Like"]
like_notnum = pasta1[(pasta1["Like"].str.contains("좋아합니")|pasta1["Like"].str.contains("눌러보세"))]["Like"]
# like_notnum = num.astype(int)

In [31]:
like_num = like_num.astype("int")
like_num.info()
like_num.median()

17.0

In [33]:
pasta1["Like"].str.contains("좋아합니").value_counts()

False    1230
True      287
Name: Like, dtype: int64

In [34]:
pasta1["Like"].str.contains("눌러보세").value_counts()

False    1459
True       58
Name: Like, dtype: int64

In [51]:
like_str_idx = pasta1[pasta1["Like"].str.contains("좋아합니") == True].index
like_0_idx = pasta1[pasta1["Like"].str.contains("눌러보세") == True].index

In [54]:
for i in range(len(pasta1)):
    for j in like_str_idx:
        for k in like_0_idx:
            if i == j:
                pasta1["Like"][j] = 17.0
            if i == k:
                pasta1["Like"][k] = 0
            else:
                continue

In [55]:
pasta1["Like"].str.contains("좋아합니").value_counts()

False    1172
Name: Like, dtype: int64

In [56]:
pasta1["Like"].str.contains("눌러보세").value_counts()

False    1172
Name: Like, dtype: int64

In [58]:
pasta1["Like"] = pasta1["Like"].astype("int")

In [59]:
pasta1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1518 entries, 0 to 1517
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Contents  1518 non-null   object
 1   Like      1518 non-null   int32 
 2   ID        1518 non-null   object
 3   Tags      1518 non-null   object
dtypes: int32(1), object(3)
memory usage: 85.7+ KB


In [61]:
pasta1.head()

,Contents,Like,ID,Tags
0,새우장 키타라 파스타 집에서 냉동새우로 새우장을 담가봤어요 제가 만들었지만 너무 맛...,204,jooboossong,"#우리의식탁, #라이프스타일러, #파스타, #파스타레시피, #파스타만들기, #새우장..."
1,chef_seokdeok안녕하세요 이태리에서 요리학교를 나와 지금까지 이태리...,17,jyourpso1028,"#파스타, #생면파스타, #연남맛집, #연남동맛집, #홍대맛집, #연트럴파크맛집, ..."
2,chef_seokdeok안녕하세요 이태리에서 요리학교를 나와 지금까지 이태리...,17,uoorspn,"#파스타, #생면파스타, #연남맛집, #연남동맛집, #홍대맛집, #연트럴파크맛집, ..."
3,chef_seokdeok안녕하세요 이태리에서 요리학교를 나와 지금까지 이태리...,17,jsytxmap,"#파스타, #생면파스타, #연남맛집, #연남동맛집, #홍대맛집, #연트럴파크맛집, ..."
4,chef_seokdeok안녕하세요 이태리에서 요리학교를 나와 지금까지 이태리...,17,jpskanwa0118,"#파스타, #생면파스타, #연남맛집, #연남동맛집, #홍대맛집, #연트럴파크맛집, ..."


In [63]:
pasta1.to_csv("./data/result_pre1_pasta.csv")

In [64]:
r = [['988', '3.7만'],
 ['702', '2.3만'],
 ['237', '2.6만'],
 ['481', '2.7만'],
 ['130', '2.1만'],
 ['189', '1만'],
 ['316', '1.8만'],
 ['319', '2.3만'],
 ['159', '1.8만'],
 ['96', '209']]

In [65]:
pd.DataFrame(r)

,0,1
0,988,3.7만
1,702,2.3만
2,237,2.6만
3,481,2.7만
4,130,2.1만
5,189,1만
6,316,1.8만
7,319,2.3만
8,159,1.8만
9,96,209
